In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!pip install timm albumentations

In [3]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import torchvision.transforms as transforms
import timm
import os
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings('ignore')

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [4]:
train_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomRotation(10),
    transforms.ColorJitter(brightness=0.1, contrast=0.1),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

val_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])


In [5]:
class imagedataset(Dataset):
    def __init__(self,df,img_dir,transform=None,is_test=False):
        self.df = df.copy()
        self.img_dir = img_dir
        self.transform = transform
        self.is_test = is_test
        if not is_test:
            label_columns = self.df.columns[1:]
            for col in label_columns:
                self.df[col] = pd.to_numeric(self.df[col],errors='coerce')
            self.df[label_columns] = self.df[label_columns].fillna(0)
            self.df[label_columns] = self.df[label_columns].astype(float)
    def __len__(self):
        return len(self.df)
        
    def __getitem__(self, idx):
        img_name = self.df.iloc[idx, 0]
        img_path = os.path.join(self.img_dir, img_name)
        image = Image.open(img_path).convert('RGB')
        if self.transform:
            image = self.transform(image)
        if self.is_test:
            return image
        else:
            labels = self.df.iloc[idx, 1:].values.astype(np.float32)
            return image, torch.FloatTensor(labels)

In [6]:
train_df = pd.read_csv('/kaggle/input/individual-test-chest-disease-detection/train.csv')
train_df

,filename,Atelectasis,Cardiomegaly,Consolidation,Edema,Enlarged Cardiomediastinum,Fracture,Lung Lesion,Lung Opacity,No Finding,Pleural Effusion,Pleural Other,Pneumonia,Pneumothorax
0,cxr00004.jpg,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1,cxr00005.jpg,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,cxr00006.jpg,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0
3,cxr00009.jpg,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,cxr00012.jpg,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9958,cxr12465.jpg,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
9959,cxr12466.jpg,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
9960,cxr12467.jpg,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
9961,cxr12468.jpg,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0


In [7]:
train_df.isnull().sum()

filename                      0
Atelectasis                   0
Cardiomegaly                  0
Consolidation                 0
Edema                         0
Enlarged Cardiomediastinum    0
Fracture                      0
Lung Lesion                   0
Lung Opacity                  0
No Finding                    0
Pleural Effusion              0
Pleural Other                 0
Pneumonia                     0
Pneumothorax                  0
dtype: int64

In [8]:
train_data,val_data = train_test_split(train_df,test_size=0.2,random_state=42)
train_data = train_data.reset_index(drop=True)
val_data = val_data.reset_index(drop=True)

In [9]:
train_dataset = imagedataset(train_data,'/kaggle/input/individual-test-chest-disease-detection/images/images',train_transform,is_test=False)
val_dataset = imagedataset(val_data,'/kaggle/input/individual-test-chest-disease-detection/images/images',val_transform,is_test=False)

In [10]:
train_dataset.__len__()

7970

In [11]:
print(train_dataset.__getitem__(1))

(tensor([[[-2.0837, -2.0837, -2.0837,  ..., -2.0837, -2.0837, -2.0837],
         [-2.0837, -2.0837, -2.0837,  ..., -2.0837, -2.0837, -2.0837],
         [-2.0837, -2.0837, -2.0837,  ..., -2.0837, -2.0837, -2.0837],
         ...,
         [-2.0837, -2.0837, -2.0837,  ..., -2.0837, -2.0837, -2.0837],
         [-2.0837, -2.0837, -2.0837,  ..., -2.0837, -2.0837, -2.0837],
         [-2.0837, -2.0837, -2.0837,  ..., -2.0837, -2.0837, -2.0837]],

        [[-2.0007, -2.0007, -2.0007,  ..., -2.0007, -2.0007, -2.0007],
         [-2.0007, -2.0007, -2.0007,  ..., -2.0007, -2.0007, -2.0007],
         [-2.0007, -2.0007, -2.0007,  ..., -2.0007, -2.0007, -2.0007],
         ...,
         [-2.0007, -2.0007, -2.0007,  ..., -2.0007, -2.0007, -2.0007],
         [-2.0007, -2.0007, -2.0007,  ..., -2.0007, -2.0007, -2.0007],
         [-2.0007, -2.0007, -2.0007,  ..., -2.0007, -2.0007, -2.0007]],

        [[-1.7696, -1.7696, -1.7696,  ..., -1.7696, -1.7696, -1.7696],
         [-1.7696, -1.7696, -1.7696,  ..., -

In [12]:
!pip install efficientnet_pytorch

  Preparing metadata (setup.py) ... done
  Created wheel for efficientnet_pytorch: filename=efficientnet_pytorch-0.7.1-py3-none-any.whl size=16426 sha256=873c1e303f1bb43bbbc70a93b564a1626f8ba3e78ff75c2074fa955e295ae9f0
  Stored in directory: /root/.cache/pip/wheels/8b/6f/9b/231a832f811ab6ebb1b32455b177ffc6b8b1cd8de19de70c09
Successfully built efficientnet_pytorch


In [13]:
# from efficientnet_pytorch import EfficientNet
# class EfficientNetChestXray(nn.Module):
#     def __init__(self, num_classes=13):
#         super().__init__()
#         self.backbone = EfficientNet.from_pretrained('efficientnet-b6')
#         self.backbone._fc = nn.Linear(self.backbone._fc.in_features, num_classes)
        
#     def forward(self, x):
#         return torch.sigmoid(self.backbone(x))

# model = EfficientNetChestXray(num_classes=13).to(device)


Downloading: "https://github.com/lukemelas/EfficientNet-PyTorch/releases/download/1.0/efficientnet-b6-c76e70fd.pth" to /root/.cache/torch/hub/checkpoints/efficientnet-b6-c76e70fd.pth
100%|██████████| 165M/165M [00:01<00:00, 126MB/s]  


Loaded pretrained weights for efficientnet-b6


In [41]:
from efficientnet_pytorch import EfficientNet
class EfficientNetChestXray(nn.Module):
    def __init__(self, num_classes=13):
        super().__init__()
        self.backbone = EfficientNet.from_pretrained('efficientnet-b6')
        self.backbone._fc = nn.Linear(self.backbone._fc.in_features, 4608)
        self.output = nn.Linear(4608,num_classes)
        # self.drop = nn.Dropout(0.25)
        
    def forward(self, x):
        x = self.backbone(x)
        return torch.sigmoid(self.output(x))

model = EfficientNetChestXray(num_classes=13).to(device)


Loaded pretrained weights for efficientnet-b6


In [42]:
model

EfficientNetChestXray(
  (backbone): EfficientNet(
    (_conv_stem): Conv2dStaticSamePadding(
      3, 56, kernel_size=(3, 3), stride=(2, 2), bias=False
      (static_padding): ZeroPad2d((0, 1, 0, 1))
    )
    (_bn0): BatchNorm2d(56, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
    (_blocks): ModuleList(
      (0): MBConvBlock(
        (_depthwise_conv): Conv2dStaticSamePadding(
          56, 56, kernel_size=(3, 3), stride=[1, 1], groups=56, bias=False
          (static_padding): ZeroPad2d((1, 1, 1, 1))
        )
        (_bn1): BatchNorm2d(56, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
        (_se_reduce): Conv2dStaticSamePadding(
          56, 14, kernel_size=(1, 1), stride=(1, 1)
          (static_padding): Identity()
        )
        (_se_expand): Conv2dStaticSamePadding(
          14, 56, kernel_size=(1, 1), stride=(1, 1)
          (static_padding): Identity()
        )
        (_project_conv): Conv2dStat

In [43]:
train_loader = DataLoader(train_dataset,batch_size=16,shuffle=True,num_workers=0)
val_loader = DataLoader(val_dataset,batch_size=16,shuffle=True,num_workers=0)

In [44]:
criterion = nn.BCELoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4, weight_decay=1e-5)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=20)

num_epochs = 5
best_val_loss = float('inf')


In [45]:
for epoch in range(num_epochs):
    model.train()
    train_loss = 0
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)

        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()

        if batch_idx % 50 == 0:
            print(f'Epoch {epoch+1}/{num_epochs}, Batch {batch_idx}, Loss: {loss.item():.4f}')

    model.eval()
    val_loss = 0
    with torch.no_grad():
        for data, target in val_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            val_loss += criterion(output, target).item()

    train_loss /= len(train_loader)
    val_loss /= len(val_loader)

    print(f'Epoch {epoch+1}: Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}')

    if val_loss < best_val_loss:
        best_val_loss = val_loss
        torch.save(model.state_dict(), 'best_model.pth')
        print(f'New best model saved with val loss: {val_loss:.4f}')

    scheduler.step()

Epoch 1/5, Batch 0, Loss: 0.6969
Epoch 1/5, Batch 50, Loss: 0.3916
Epoch 1/5, Batch 100, Loss: 0.3409
Epoch 1/5, Batch 150, Loss: 0.3395
Epoch 1/5, Batch 200, Loss: 0.3237
Epoch 1/5, Batch 250, Loss: 0.3381
Epoch 1/5, Batch 300, Loss: 0.3250
Epoch 1/5, Batch 350, Loss: 0.2663
Epoch 1/5, Batch 400, Loss: 0.3088
Epoch 1/5, Batch 450, Loss: 0.3097
Epoch 1: Train Loss: 0.3358, Val Loss: 0.2973
New best model saved with val loss: 0.2973
Epoch 2/5, Batch 0, Loss: 0.2436
Epoch 2/5, Batch 50, Loss: 0.3162
Epoch 2/5, Batch 100, Loss: 0.2951
Epoch 2/5, Batch 150, Loss: 0.3675
Epoch 2/5, Batch 200, Loss: 0.3152
Epoch 2/5, Batch 250, Loss: 0.2712
Epoch 2/5, Batch 300, Loss: 0.3237
Epoch 2/5, Batch 350, Loss: 0.3149
Epoch 2/5, Batch 400, Loss: 0.3155
Epoch 2/5, Batch 450, Loss: 0.3792
Epoch 2: Train Loss: 0.2958, Val Loss: 0.2866
New best model saved with val loss: 0.2866
Epoch 3/5, Batch 0, Loss: 0.3061
Epoch 3/5, Batch 100, Loss: 0.2341
Epoch 3/5, Batch 150, Loss: 0.2974
Epoch 3/5, Batch 200, Los

KeyboardInterrupt: 

In [ ]:
test_df = pd.read_csv('/kaggle/input/individual-test-chest-disease-detection/test_submission.csv')
test_dataset = imagedataset(test_df, '/kaggle/input/individual-test-chest-disease-detection/images/images', val_transform, is_test=True)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False, num_workers=2)

In [21]:
pred = []
with torch.no_grad():
    for batch_idx,data in enumerate(test_loader):
        data = data.to(device)
        output = model(data)
        pred.extend(output.cpu().numpy())

pred = np.array(pred)
pred

array([[0.8046182 , 0.5339917 , 0.0626923 , ..., 0.01952877, 0.06956998,
        0.10400461],
       [0.4305566 , 0.45760694, 0.0815184 , ..., 0.0834487 , 0.10002819,
        0.07633375],
       [0.29041103, 0.4887438 , 0.03601116, ..., 0.14502907, 0.07726175,
        0.03714387],
       ...,
       [0.1366944 , 0.06993024, 0.0996615 , ..., 0.03282491, 0.5130245 ,
        0.00942439],
       [0.45299336, 0.09918592, 0.05918641, ..., 0.0804657 , 0.10270098,
        0.1028093 ],
       [0.31483936, 0.23656711, 0.21495116, ..., 0.08976375, 0.2019137 ,
        0.3358281 ]], dtype=float32)

In [23]:
test_df

,filename,Atelectasis,Cardiomegaly,Consolidation,Edema,Enlarged Cardiomediastinum,Fracture,Lung Lesion,Lung Opacity,No Finding,Pleural Effusion,Pleural Other,Pneumonia,Pneumothorax
0,cxr00001.jpg,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1,cxr00002.jpg,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,cxr00003.jpg,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3,cxr00007.jpg,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,cxr00008.jpg,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2501,cxr08417.jpg,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2502,cxr08428.jpg,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2503,cxr09558.jpg,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2504,cxr10456.jpg,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [24]:
columns = ['Atelectasis', 'Cardiomegaly', 'Consolidation', 'Edema',
           'Enlarged Cardiomediastinum', 'Fracture', 'Lung Lesion',
           'Lung Opacity', 'No Finding', 'Pleural Effusion',
           'Pleural Other', 'Pneumonia', 'Pneumothorax']

submission = pd.DataFrame(pred,columns=columns)
submission.insert(0,'filename',test_df['filename'])

submission.to_csv('sub_Chest_Disease_Detectio_2.csv', index=False)

In [27]:
submission

,filename,Atelectasis,Cardiomegaly,Consolidation,Edema,Enlarged Cardiomediastinum,Fracture,Lung Lesion,Lung Opacity,No Finding,Pleural Effusion,Pleural Other,Pneumonia,Pneumothorax
0,cxr00001.jpg,0.804618,0.533992,0.062692,0.157346,0.114438,0.028024,0.043732,0.402336,0.010410,0.965280,0.019529,0.069570,0.104005
1,cxr00002.jpg,0.430557,0.457607,0.081518,0.104704,0.174090,0.068204,0.098548,0.296539,0.069689,0.743733,0.083449,0.100028,0.076334
2,cxr00003.jpg,0.290411,0.488744,0.036011,0.139538,0.166651,0.200075,0.080203,0.223290,0.194081,0.511766,0.145029,0.077262,0.037144
3,cxr00007.jpg,0.063005,0.401852,0.109071,0.641778,0.031983,0.026797,0.095814,0.348766,0.080116,0.091817,0.054719,0.362000,0.005641
4,cxr00008.jpg,0.119053,0.303543,0.015774,0.198727,0.028667,0.026334,0.065497,0.145246,0.371442,0.032380,0.011831,0.071492,0.001384
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2501,cxr08417.jpg,0.004200,0.005745,0.004575,0.000511,0.050361,0.038927,0.020497,0.038628,0.840689,0.001036,0.007031,0.046748,0.061682
2502,cxr08428.jpg,0.096230,0.014943,0.421301,0.148986,0.004015,0.002188,0.171469,0.534593,0.007348,0.110651,0.005543,0.821242,0.001671
2503,cxr09558.jpg,0.136694,0.069930,0.099661,0.097285,0.038859,0.093909,0.163395,0.395080,0.371355,0.020510,0.032825,0.513025,0.009424
2504,cxr10456.jpg,0.452993,0.099186,0.059186,0.014484,0.126676,0.233653,0.173453,0.272265,0.201296,0.334986,0.080466,0.102701,0.102809


In [ ]:
label_columns = submission.columns[1:]

submission_binary = submission.copy()
# submission_binary[label_columns] = (submission_binary[label_columns] >= 0.5).astype(int) # 0.42818
# submission_binary[label_columns] = (submission_binary[label_columns] >= 0.4).astype(int) # 0.48
# submission_binary[label_columns] = (submission_binary[label_columns] >= 0.37).astype(int) # 0.519
# submission_binary[label_columns] = (submission_binary[label_columns] >= 0.34).astype(int) # 0.517
# submission_binary[label_columns] = (submission_binary[label_columns] >= 0.355).astype(int) # 0.519 : private : 0.53189
submission_binary[label_columns] = (submission_binary[label_columns] >= 0.362).astype(int) # private : 0.53275
submission_binary.to_csv('sub_Chest_Disease_Detectio_7_binary.csv', index=False)